In [1]:
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [8]:
# name = '450_50'
name_of_dataset = "datasets/240_256_dataset_main.npz"

In [9]:
def load_data(name):
    """Возвращает массиы с данными из npz по имени"""
    data = np.load(name)
    return data['x_train'], data['x_test'], data['y_train'], data['y_test']

In [10]:
x_train, x_test, y_train, y_test = load_data(name_of_dataset)

y_train_in = to_categorical(y_train, 2)
x_train_in = x_train / 255
y_test_in = to_categorical(y_test, 2)
x_test_in = x_test / 255

x_train_in = np.expand_dims(x_train, axis=3)
x_test_in = np.expand_dims(x_test, axis=3)


In [11]:
checkpoint_path = "trained_reseek/first_trained.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
model = keras.Sequential([
        Flatten(input_shape=(240, 256, 1)),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#     print(model.summary())

epochs = 50
#     start_fit = time.time()
history = model.fit(x_train_in, y_train_in, batch_size=32, epochs=epochs, validation_split=0.2, callbacks=[cp_callback])
#     end_fit = time.time()

#     start_eval = time.time()
model.evaluate(x_test_in, y_test_in)
#     end_eval = time.time()
# model.save("train_fst.h5")

Train on 2240 samples, validate on 560 samples
Epoch 1/50
2208/2240 [============================>.] - ETA: 0s - loss: 189.5300 - acc: 0.6938
Epoch 00001: saving model to trained_reseek/first_trained.ckpt
2240/2240 [==============================] - 11s 5ms/sample - loss: 187.1077 - acc: 0.6951 - val_loss: 23.7499 - val_acc: 0.7536
Epoch 2/50
2208/2240 [============================>.] - ETA: 0s - loss: 10.2515 - acc: 0.7812
Epoch 00002: saving model to trained_reseek/first_trained.ckpt
2240/2240 [==============================] - 11s 5ms/sample - loss: 10.1104 - acc: 0.7821 - val_loss: 1.7726 - val_acc: 0.7964
Epoch 3/50
2208/2240 [============================>.] - ETA: 0s - loss: 1.0713 - acc: 0.7849
Epoch 00003: saving model to trained_reseek/first_trained.ckpt
2240/2240 [==============================] - 11s 5ms/sample - loss: 1.0619 - acc: 0.7853 - val_loss: 1.5685 - val_acc: 0.7893
Epoch 4/50
2208/2240 [============================>.] - ETA: 0s - loss: 1.0773 - acc: 0.7880
Epoch 0

Epoch 30/50
2208/2240 [============================>.] - ETA: 0s - loss: 0.5097 - acc: 0.7464
Epoch 00030: saving model to trained_reseek/first_trained.ckpt
2240/2240 [==============================] - 10s 4ms/sample - loss: 0.5087 - acc: 0.7473 - val_loss: 1.7490 - val_acc: 0.7589
Epoch 31/50
2208/2240 [============================>.] - ETA: 0s - loss: 0.6673 - acc: 0.7763
Epoch 00031: saving model to trained_reseek/first_trained.ckpt
2240/2240 [==============================] - 11s 5ms/sample - loss: 0.6622 - acc: 0.7763 - val_loss: 2.3418 - val_acc: 0.8054
Epoch 32/50
2208/2240 [============================>.] - ETA: 0s - loss: 0.5999 - acc: 0.7613
Epoch 00032: saving model to trained_reseek/first_trained.ckpt
2240/2240 [==============================] - 10s 5ms/sample - loss: 0.5980 - acc: 0.7603 - val_loss: 1.5056 - val_acc: 0.7696
Epoch 33/50
2208/2240 [============================>.] - ETA: 0s - loss: 0.5261 - acc: 0.7545
Epoch 00033: saving model to trained_reseek/first_trained

[0.95007798910141, 0.59]

In [52]:
os.listdir(checkpoint_dir)


['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [ ]:

# plt.plot(history.history['accuracy'], label='на обучающих данных')
# plt.plot(history.history['val_accuracy'], label='на тестовых')
# plt.title('Точность')
# plt.xlabel('Эпоха')
# plt.ylabel('Значене')
# plt.legend()
# plt.show()


In [56]:
model2.load_weights(checkpoint_path)

In [33]:
TP, FP, TN, FN = 0, 0, 0, 0
error_log = []
for i in range(len(x_test)):
    qwe = np.expand_dims(x_test[i], axis=0)
    img_expended = np.expand_dims(qwe, axis=3)
    prediction = model.predict(img_expended)[0]
    if prediction[1] > prediction[0]:
        title = 'цель'
        if y_test[i] == 1:
            TP += 1 
        else:
            FN += 1
            error_log.append(str(i) + '_defined_as_target')
    else:
        title = 'помехи'
        if y_test[i] == 1:
            FP += 1 
            error_log.append(str(i) + '_defined_as_stray')
        else:
            TN += 1
#     plt.figure()
#     plt.imshow(x_test[i])
#     plt.title(f'{title}')

precisition = round(TP/(TP + FP), 2)
recall = round(TP/(TP + FN), 2)

targets_in_train = 0
targets_in_test = 0
for i in y_train:
    if i == 1:
        targets_in_train += 1
for i in y_test:
    if i == 1:
        targets_in_test += 1
        
train = len(y_train)
test = len(y_test)
studying_time = round(end_fit - start_fit, 2)
prediction_time = round((end_eval - start_eval)/len(y_test_in), 2)

detection_probability = round(TP/targets_in_test, 2)
false_alarm_probability = round(FN/(test - targets_in_test), 2)
print(f'Данные для набора из {train} подматриц'
     f'\n{test} -- для испытания'
     f'\n{targets_in_train} целей и {train - targets_in_train} помех'
     f'\nСобрано на испытательном наборе ({test})')
print('\nВремя обучения -->', studying_time,
     '\nКоличество эпох (итераций обучения) -->', epochs,
     '\nВремя предсказания для одной подматрицы -->', prediction_time)
print('\n', TP, '-- отгаданные цели'
     '\n', FP, '-- не отгаданные цели'
     '\n', TN, '-- отгаданные помехи'
     '\n', FN, '-- не отгаданные помехи (ошибочные цели) --> ложная тревога'
     '\n', precisition, '-- доля верных распознаний (precisition)'
     '\n', recall, '-- доля верно распознанных целей (из всех целей в тестовом наборе) (recall)')
print(f'P обнаружения --> {detection_probability}'
     f'\nP ложной тревоги --> {false_alarm_probability}')


temp_list = [name_of_dataset, train, targets_in_train, train - targets_in_train, studying_time, 
             detection_probability, false_alarm_probability]
with open('table_data_fst.txt', 'a') as file:
    print(*temp_list, file=file)
with open(name + '.txt', 'w') as file:
    print(*error_log, file=file, sep='\n')

Данные для набора из 500 подматриц
200 -- для испытания
450 целей и 50 помех
Собрано на испытательном наборе (200)

Время обучения --> 71.88 
Количество эпох (итераций обучения) --> 50 
Время предсказания для одной подматрицы --> 0.0

 99 -- отгаданные цели
 0 -- не отгаданные цели
 6 -- отгаданные помехи
 95 -- не отгаданные помехи (ошибочные цели) --> ложная тревога
 1.0 -- доля верных распознаний (precisition)
 0.51 -- доля верно распознанных целей (из всех целей в тестовом наборе) (recall)
P обнаружения --> 1.0
P ложной тревоги --> 0.94
